In [1]:
import sys
sys.path.append("/projects/xtb_python/") # go to parent dir

import json
import socket
import logging
import time
import ssl
from threading import Thread
import yaml
from src.xtb.xAPIConnector import *

# set to true on debug environment only
DEBUG = False
INFO = True

#default connection properites
DEFAULT_XAPI_ADDRESS        = 'xapi.xtb.com'
#DEFAULT_XAPI_PORT           = 5124  # Demo
#DEFUALT_XAPI_STREAMING_PORT = 5125  # Demo
DEFAULT_XAPI_PORT           = 5112  # Real
DEFUALT_XAPI_STREAMING_PORT = 5113  # Real

# wrapper name and version
WRAPPER_NAME    = 'python'
WRAPPER_VERSION = '2.5.0'

# API inter-command timeout (in ms)
API_SEND_TIMEOUT = 100

# max connection tries
API_MAX_CONN_TRIES = 3

# logger properties
logger = logging.getLogger("jsonSocket")
FORMAT = '[%(asctime)-15s][%(funcName)s:%(lineno)d] %(message)s'
logging.basicConfig(format=FORMAT)

# private information
USERID = ''
PASSWORD = ''
with open('../config/config.yml') as file:
    user_yml = yaml.safe_load(file)
    USERID = user_yml['user_id']
    PASSWORD = user_yml['password']

if DEBUG:
    logger.setLevel(logging.DEBUG)
elif INFO:
    logger.setLevel(logging.INFO)
else:
    logger.setLevel(logging.CRITICAL)


In [2]:
# example function for processing ticks from Streaming socket
def procTickExample(msg): 
    print("TICK: ", msg)

# example function for processing trades from Streaming socket
def procTradeExample(msg): 
    print("TRADE: ", msg)

# example function for processing trades from Streaming socket
def procBalanceExample(msg): 
    print("BALANCE: ", msg)

# example function for processing trades from Streaming socket
def procTradeStatusExample(msg): 
    print("TRADE STATUS: ", msg)

# example function for processing trades from Streaming socket
def procProfitExample(msg): 
    print("PROFIT: ", msg)

# example function for processing news from Streaming socket
def procNewsExample(msg): 
    print("NEWS: ", msg)
    

def main():

    # enter your login credentials here
    userId = USERID
    password = PASSWORD

    # create & connect to RR socket
    client = APIClient()
    
    # connect to RR socket, login
    loginResponse = client.execute(loginCommand(userId=userId, password=password))
    logger.info(str(loginResponse)) 

    # check if user logged in correctly
    if(loginResponse['status'] == False):
        print('Login failed. Error code: {0}'.format(loginResponse['errorCode']))
        return

    # get ssId from login response
    ssid = loginResponse['streamSessionId']
    
    # second method of invoking commands
    #resp = client.commandExecute('getAllSymbols')
    resp = client.commandExecute('getMarginLevel')
    
    print(1)
    # create & connect to Streaming socket with given ssID
    # and functions for processing ticks, trades, profit and tradeStatus
    sclient = APIStreamClient(ssId=ssid, tickFun=procTickExample, tradeFun=procTradeExample, profitFun=procProfitExample, tradeStatusFun=procTradeStatusExample)
    print(2)

    # subscribe for trades
    sclient.subscribeTrades()
    print(3)
    # subscribe for prices
    sclient.subscribePrices(['EURUSD', 'EURGBP', 'EURJPY'])
    print(4)
    # subscribe for profits
    sclient.subscribeProfits()
    print(5)
    # this is an example, make it run for 5 seconds
    time.sleep(5)
    print(6)
    # gracefully close streaming socket
    sclient.disconnect()
    print(7)
    # gracefully close RR socket
    client.disconnect()
    print(8)


In [3]:
main()

[2020-03-23 17:51:50,089][connect:77] Socket connected
[2020-03-23 17:51:50,091][_waitingSend:91] Sent: b'{"command": "login", "arguments": {"userId": 1546078, "password": "qhUrKfyPj!HR7Fj", "appName": ""}}'
[2020-03-23 17:51:50,380][_read:110] Received: {'status': True, 'streamSessionId': '262540fffee58c68-0000501c-001000c3-31627f555a0b4ad4-3ddf2770'}
[2020-03-23 17:51:50,380][main:37] {'status': True, 'streamSessionId': '262540fffee58c68-0000501c-001000c3-31627f555a0b4ad4-3ddf2770'}
[2020-03-23 17:51:50,384][_waitingSend:91] Sent: b'{"command": "getMarginLevel", "arguments": {}}'
[2020-03-23 17:51:50,610][_read:110] Received: {'status': True, 'returnData': {'balance': 6614.26, 'margin': 0.0, 'currency': 'CLP', 'credit': 0.0, 'equityFX': 6614.26, 'equity': 6614.26, 'margin_free': 6614.26, 'margin_level': 0.0, 'stockValue': 0.0, 'stockLock': 0.0, 'cashStockValue': 0.0}}


1


[2020-03-23 17:51:51,335][connect:77] Socket connected
[2020-03-23 17:51:51,338][_waitingSend:91] Sent: b'{"command": "getTrades", "streamSessionId": "262540fffee58c68-0000501c-001000c3-31627f555a0b4ad4-3ddf2770"}'
[2020-03-23 17:51:51,440][_waitingSend:91] Sent: b'{"command": "getTickPrices", "symbol": "EURUSD", "streamSessionId": "262540fffee58c68-0000501c-001000c3-31627f555a0b4ad4-3ddf2770"}'


2
3


[2020-03-23 17:51:51,542][_waitingSend:91] Sent: b'{"command": "getTickPrices", "symbol": "EURGBP", "streamSessionId": "262540fffee58c68-0000501c-001000c3-31627f555a0b4ad4-3ddf2770"}'
[2020-03-23 17:51:51,643][_waitingSend:91] Sent: b'{"command": "getTickPrices", "symbol": "EURJPY", "streamSessionId": "262540fffee58c68-0000501c-001000c3-31627f555a0b4ad4-3ddf2770"}'
[2020-03-23 17:51:51,738][_read:110] Received: {'command': 'tickPrices', 'data': {'symbol': 'EURUSD', 'ask': 1.07765, 'bid': 1.0773, 'high': 1.08267, 'low': 1.06565, 'askVolume': 350000, 'bidVolume': 500000, 'timestamp': 1584985911607, 'level': 0, 'quoteId': 5, 'spreadTable': 3.5, 'spreadRaw': 0.00035}}
[2020-03-23 17:51:51,739][_readStream:201] Stream received: {'command': 'tickPrices', 'data': {'symbol': 'EURUSD', 'ask': 1.07765, 'bid': 1.0773, 'high': 1.08267, 'low': 1.06565, 'askVolume': 350000, 'bidVolume': 500000, 'timestamp': 1584985911607, 'level': 0, 'quoteId': 5, 'spreadTable': 3.5, 'spreadRaw': 0.00035}}
[2020-03-

...

TICK:  {'command': 'tickPrices', 'data': {'symbol': 'EURJPY', 'ask': 119.858, 'bid': 119.77, 'high': 119.905, 'low': 117.696, 'askVolume': 1200000, 'bidVolume': 1200000, 'timestamp': 1584985912722, 'level': 3, 'quoteId': 5, 'spreadTable': 8.8, 'spreadRaw': 0.088}}
TICK:  {'command': 'tickPrices', 'data': {'symbol': 'EURJPY', 'ask': 119.87, 'bid': 119.766, 'high': 119.905, 'low': 117.696, 'askVolume': 4000000, 'bidVolume': 3500000, 'timestamp': 1584985912722, 'level': 4, 'quoteId': 5, 'spreadTable': 10.4, 'spreadRaw': 0.104}}
TICK:  {'command': 'tickPrices', 'data': {'symbol': 'EURUSD', 'ask': 1.07767, 'bid': 1.07732, 'high': 1.08267, 'low': 1.06565, 'askVolume': 300000, 'bidVolume': 350000, 'timestamp': 1584985912870, 'level': 0, 'quoteId': 5, 'spreadTable': 3.5, 'spreadRaw': 0.00035}}
TICK:  {'command': 'tickPrices', 'data': {'symbol': 'EURUSD', 'ask': 1.07768, 'bid': 1.07731, 'high': 1.08267, 'low': 1.06565, 'askVolume': 1000000, 'bidVolume': 800000, 'timestamp': 1584985912870, 'leve

[2020-03-23 17:51:56,666][_read:110] Received: {'command': 'tickPrices', 'data': {'symbol': 'EURUSD', 'ask': 1.07778, 'bid': 1.07721, 'high': 1.08267, 'low': 1.06565, 'askVolume': 3500000, 'bidVolume': 3000000, 'timestamp': 1584985912870, 'level': 3, 'quoteId': 5, 'spreadTable': 5.7, 'spreadRaw': 0.00057}}
[2020-03-23 17:51:56,667][_readStream:201] Stream received: {'command': 'tickPrices', 'data': {'symbol': 'EURUSD', 'ask': 1.07778, 'bid': 1.07721, 'high': 1.08267, 'low': 1.06565, 'askVolume': 3500000, 'bidVolume': 3000000, 'timestamp': 1584985912870, 'level': 3, 'quoteId': 5, 'spreadTable': 5.7, 'spreadRaw': 0.00057}}
[2020-03-23 17:51:56,668][_read:110] Received: {'command': 'tickPrices', 'data': {'symbol': 'EURUSD', 'ask': 1.07787, 'bid': 1.07712, 'high': 1.08267, 'low': 1.06565, 'askVolume': 4000000, 'bidVolume': 3500000, 'timestamp': 1584985912870, 'level': 4, 'quoteId': 5, 'spreadTable': 7.5, 'spreadRaw': 0.00075}}
[2020-03-23 17:51:56,670][_readStream:201] Stream received: {'c

TICK:  {'command': 'tickPrices', 'data': {'symbol': 'EURUSD', 'ask': 1.07778, 'bid': 1.07721, 'high': 1.08267, 'low': 1.06565, 'askVolume': 3500000, 'bidVolume': 3000000, 'timestamp': 1584985912870, 'level': 3, 'quoteId': 5, 'spreadTable': 5.7, 'spreadRaw': 0.00057}}
TICK:  {'command': 'tickPrices', 'data': {'symbol': 'EURUSD', 'ask': 1.07787, 'bid': 1.07712, 'high': 1.08267, 'low': 1.06565, 'askVolume': 4000000, 'bidVolume': 3500000, 'timestamp': 1584985912870, 'level': 4, 'quoteId': 5, 'spreadTable': 7.5, 'spreadRaw': 0.00075}}
TICK:  {'command': 'tickPrices', 'data': {'symbol': 'EURGBP', 'ask': 0.93317, 'bid': 0.93239, 'high': 0.93845, 'low': 0.9154, 'askVolume': 250000, 'bidVolume': 250000, 'timestamp': 1584985912919, 'level': 0, 'quoteId': 5, 'spreadTable': 7.8, 'spreadRaw': 0.00078}}
6


[2020-03-23 17:51:56,871][_read:110] Received: {'command': 'tickPrices', 'data': {'symbol': 'EURGBP', 'ask': 0.93319, 'bid': 0.93237, 'high': 0.93845, 'low': 0.9154, 'askVolume': 700000, 'bidVolume': 900000, 'timestamp': 1584985912919, 'level': 1, 'quoteId': 5, 'spreadTable': 8.2, 'spreadRaw': 0.00082}}
[2020-03-23 17:51:56,873][_readStream:201] Stream received: {'command': 'tickPrices', 'data': {'symbol': 'EURGBP', 'ask': 0.93319, 'bid': 0.93237, 'high': 0.93845, 'low': 0.9154, 'askVolume': 700000, 'bidVolume': 900000, 'timestamp': 1584985912919, 'level': 1, 'quoteId': 5, 'spreadTable': 8.2, 'spreadRaw': 0.00082}}


TICK:  {'command': 'tickPrices', 'data': {'symbol': 'EURGBP', 'ask': 0.93319, 'bid': 0.93237, 'high': 0.93845, 'low': 0.9154, 'askVolume': 700000, 'bidVolume': 900000, 'timestamp': 1584985912919, 'level': 1, 'quoteId': 5, 'spreadTable': 8.2, 'spreadRaw': 0.00082}}
7
8
